In [1]:
%load_ext autoreload
%autoreload 2

## Shape analysis

The goal is to have a generalized algorithm that for all symmetrically equivalent cluster centers, find nearest atoms from trajectory, and transforms them back to the asymmetric unit. This helps the statistics for performing shape analysis and making plots.

As input:

1. crystal or material structure
    - contains cluster centers
    - includes symmetry operations
2. trajectory
    - typically P1
    - maybe a supercell of crystal structure
    - lattice can be triclinic (non-constrained in simulation)

Algorithm:

1. load clusters (pymatgen structure)
2. load trajectory (pymatgen trajectory)
3. reduce supercell of trajectory to match clusters
    - assert trajectory and cluster lattices match
4. for every unique cluster center, 
    5. for every symmetry operation
    - apply next symmetry operation to cluster center
    - find all trajectory points within X distance
    - copy and map points back to asymmetric unit (reverse symmetry op)
    - subtract cluster center coords
10. perform shape analysis
    - plots, fits, heat maps, msd, etc
   
See: https://github.com/GEMDAT-repos/GEMDAT/pull/166

In [2]:
from __future__ import annotations

from pathlib import Path

import numpy as np

from gemdat import Trajectory
from gemdat.io import load_known_material

trajectory = Trajectory.from_vasprun(
    Path(
        '/home/stef/md-analysis-matlab-example-short/shape_analysis/vasprun.xml'
    ))

supercell = (2, 1, 1)

diff_trajectory = trajectory.filter('Li')

structure = load_known_material('argyrodite')

/home/stef/python/gemdat/.venv/lib64/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [3]:
diff_trajectory.get_lattice(0)

Lattice
    abc : 19.84771 9.923855 9.923855
 angles : 90.0 90.0 90.0
 volume : 1954.6600006747865
      A : 19.84771 0.0 0.0
      B : 0.0 9.923855 0.0
      C : 0.0 0.0 9.923855
    pbc : True True True

In [4]:
# collapse coords
positions = diff_trajectory.positions.reshape(-1, 3)
positions, positions.shape

(array([[0.16145033, 0.33547197, 0.02452775],
        [0.10760483, 0.71007687, 0.50529059],
        [0.37023313, 0.71662174, 0.00237432],
        ...,
        [0.5081269 , 0.69439527, 0.7062491 ],
        [0.91863445, 0.47414316, 0.64892594],
        [0.00493453, 0.24402231, 0.67557806]]),
 (240000, 3))

In [5]:
# collapse supercell
positions = np.mod(positions, 1 / np.array(supercell)) * np.array(supercell)
positions, positions.shape

(array([[0.32290066, 0.33547197, 0.02452775],
        [0.21520966, 0.71007687, 0.50529059],
        [0.74046626, 0.71662174, 0.00237432],
        ...,
        [0.0162538 , 0.69439527, 0.7062491 ],
        [0.8372689 , 0.47414316, 0.64892594],
        [0.00986906, 0.24402231, 0.67557806]]),
 (240000, 3))

In [6]:
from pymatgen import symmetry

sga = symmetry.analyzer.SpacegroupAnalyzer(structure)

symops = sga.get_space_group_operations()
symstruct = sga.get_symmetrized_structure()
lattice = symstruct.lattice
lattice

Lattice
    abc : 9.924 9.924 9.924
 angles : 90.0 90.0 90.0
 volume : 977.3728410239999
      A : 9.924 0.0 6.076697417369166e-16
      B : 1.5959009175390938e-15 9.924 6.076697417369166e-16
      C : 0.0 0.0 9.924
    pbc : True True True

## TODO

- [ ] Loop over unique coords
- [ ] Store cluster per unique label
- [ ] plots

In [7]:
thresh = 1.0  # Å

cluster = []

unq_coord = symstruct[0].frac_coords

print(unq_coord)

for op in symops:
    cluster_coord = op.operate(unq_coord)
    dists = lattice.get_all_distances(cluster_coord, positions)

    sel = dists < thresh
    close = positions[sel.flatten()]

    # digitize differences to move all close positions to
    # same sphere around cluster center
    offsets = np.digitize(close - cluster_coord, bins=[0.5, -0.4999999]) - 1

    close += offsets

    print(close.size)
    print('{: .3f} {: .3f} {: .3f}'.format(*close.mean(axis=0)))

    x1, x2 = close.min(axis=0), close.max(axis=0)
    print(
        '{: .3f} {: .3f} {: .3f} | {: .3f} {: .3f} {: .3f} | {: .3f} {: .3f} {: .3f}'
        .format(*x1, *x2, *(x2 - x1)))

    inversed = op.inverse.operate_multi(close)

    print('{: .3f} {: .3f} {: .3f}'.format(*inversed.mean(axis=0)))

    x1, x2 = inversed.min(axis=0), inversed.max(axis=0)
    print(
        '{: .3f} {: .3f} {: .3f} | {: .3f} {: .3f} {: .3f} | {: .3f} {: .3f} {: .3f}'
        .format(*x1, *x2, *(x2 - x1)))

    cluster.append(inversed)

[0.183 0.183 0.024]
13077
 0.201  0.186  0.017
 0.091  0.089 -0.044 |  0.272  0.276  0.112 |  0.181  0.187  0.156
 0.201  0.186  0.017
 0.091  0.089 -0.044 |  0.272  0.276  0.112 |  0.181  0.187  0.156
14025
 0.204 -0.184 -0.029
 0.085 -0.275 -0.100 |  0.281 -0.091  0.054 |  0.197  0.183  0.154
 0.184  0.204  0.029
 0.091  0.085 -0.054 |  0.275  0.281  0.100 |  0.183  0.197  0.154
16341
-0.203 -0.181  0.026
-0.280 -0.277 -0.065 | -0.122 -0.083  0.098 |  0.158  0.194  0.163
 0.203  0.181  0.026
 0.122  0.083 -0.065 |  0.280  0.277  0.098 |  0.158  0.194  0.163
16035
-0.191  0.188 -0.035
-0.279  0.087 -0.113 | -0.092  0.282  0.067 |  0.187  0.195  0.179
 0.188  0.191  0.035
 0.087  0.092 -0.067 |  0.282  0.279  0.113 |  0.195  0.187  0.179
14025
 0.204 -0.184 -0.029
 0.085 -0.275 -0.100 |  0.281 -0.091  0.054 |  0.197  0.183  0.154
 0.204  0.184  0.029
 0.085  0.091 -0.054 |  0.281  0.275  0.100 |  0.197  0.183  0.154
13077
 0.201  0.186  0.017
 0.091  0.089 -0.044 |  0.272  0.276  0.112